# Evaluation DIKI small

In [1]:
import pandas as pd

#### Read DIKI Small

In [2]:
df = pd.read_csv("DIKI_small.csv")

In [3]:
df.head()

,key words
0,"""absurdistan"""
1,"""bild-dungsbürger"""
2,"""eierlosen"""
3,"""entsorgen"""
4,"""entsorgt"""


In [4]:
len(df)

2874

In [5]:
low_incivility_unigrams = list(df["key words"])

In [6]:
low_incivility_unigrams[:30]

['"absurdistan"',
 '"bild-dungsbürger"',
 '"eierlosen"',
 '"entsorgen"',
 '"entsorgt"',
 '"flüchtling"',
 '"flüchtlinge"',
 '"friedenstaube"',
 '"gästen"',
 '"heimkehrer"',
 '"integrieren"',
 '"männergruppen"',
 '"pirat"',
 '"schiffbrüchigen"',
 '"schutzberechtigten"',
 '"sozialpopulist"',
 '"spitzenmann"',
 '"wähler"',
 '"zecken"',
 '"ziegenhirten"',
 '#abgefucktespd',
 '#abschiebeindustrie',
 '#afdnazis',
 '#afdwegfegen',
 '#afdwuergt',
 '#afdwürgt',
 '#alkoholiker',
 '#allahkotz',
 '#altparteien',
 '#altparteienkartel']

In [7]:
len(low_incivility_unigrams)

2874

In [8]:
len(set(low_incivility_unigrams)) #no duplicates

2874

#### Load Test Data

In [9]:
df_test = pd.read_csv("GermEval21_TestData.csv")

In [13]:
df_test.head()

,comment_id,comment_text,Sub1_Toxic,Sub2_Engaging,Sub3_FactClaiming
0,3245,@USER Sie würden wahrscheinlich auch einen Kri...,1,1,0
1,3246,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,1,1
2,3247,@USER: Man kann natürlich immerzu dieselbe Sau...,0,1,1
3,3248,"Als jemand, der im real existierenden Sozialis...",1,0,0
4,3249,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,0,0


In [11]:
len(df_test)

944

In [12]:
df_test["Sub1_Toxic"].value_counts()

0    594
1    350
Name: Sub1_Toxic, dtype: int64

In [14]:
df_test = df_test[["comment_text", "Sub1_Toxic"]] #Keep only the columns needed.

In [15]:
df_test.head()

,comment_text,Sub1_Toxic
0,@USER Sie würden wahrscheinlich auch einen Kri...,1
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0
3,"Als jemand, der im real existierenden Sozialis...",1
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1


### Matching method isin

In [16]:
def match_unigrams_isin(X):
    match = []
    X = str(X)  # vorsichtshalber

    X = str.lower(X)



    for i in low_incivility_unigrams:
        if i in X:
            match.append(i)
           # print(i)

    return (match)

In [17]:
df_test["Matches_isin"]=df_test["comment_text"].apply(lambda x: match_unigrams_isin(x))

In [18]:
df_test

,comment_text,Sub1_Toxic,Matches_isin
0,@USER Sie würden wahrscheinlich auch einen Kri...,1,[]
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,[]
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0,"[alte, amigo, esel, hund, hunde, linke, sau]"
3,"Als jemand, der im real existierenden Sozialis...",1,"[sozi, trottel]"
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,[]
...,...,...,...
939,@USER Da ist wie gegen Windmühlen kämpfen,0,[]
940,@USER Ihr Wort in Gottes Ohr.,0,[]
941,😂,0,[😂]
942,Mimimimi,1,"[mimimi, mimimimi]"


In [19]:
df_test["Number_Matches_isin"] = df_test["Matches_isin"].apply(lambda x: len(x))

In [20]:
def matches_to_sting(X):

    X = ", ".join(X)

    return (X)

In [21]:
df_test["Matches_isin_str"] = df_test["Matches_isin"].apply(lambda x: matches_to_sting(x))

In [22]:
def f(row):
    if row['Number_Matches_isin'] > 0:
        val = 1
    elif row['Number_Matches_isin'] == 0:
        val = 0
    
    return val

In [23]:
df_test['label_isin_dich']  = df_test.apply(f, axis=1)

In [24]:
df_test

,comment_text,Sub1_Toxic,Matches_isin,Number_Matches_isin,Matches_isin_str,label_isin_dich
0,@USER Sie würden wahrscheinlich auch einen Kri...,1,[],0,,0
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,[],0,,0
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0,"[alte, amigo, esel, hund, hunde, linke, sau]",7,"alte, amigo, esel, hund, hunde, linke, sau",1
3,"Als jemand, der im real existierenden Sozialis...",1,"[sozi, trottel]",2,"sozi, trottel",1
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,[],0,,0
...,...,...,...,...,...,...
939,@USER Da ist wie gegen Windmühlen kämpfen,0,[],0,,0
940,@USER Ihr Wort in Gottes Ohr.,0,[],0,,0
941,😂,0,[😂],1,😂,1
942,Mimimimi,1,"[mimimi, mimimimi]",2,"mimimi, mimimimi",1


In [25]:
df_test.label_isin_dich.value_counts()

0    517
1    427
Name: label_isin_dich, dtype: int64

#### Quantitative Error Analysis

In [26]:
from sklearn.metrics import classification_report
print(classification_report(df_test["Sub1_Toxic"], df_test["label_isin_dich"]))

              precision    recall  f1-score   support

           0       0.73      0.64      0.68       594
           1       0.49      0.60      0.54       350

    accuracy                           0.62       944
   macro avg       0.61      0.62      0.61       944
weighted avg       0.64      0.62      0.63       944



#### Qualitative Error Analysis

In [27]:
#False Positives
df_fp = df_test[["comment_text","Matches_isin_str"]][df_test["Sub1_Toxic"] < df_test["label_isin_dich"]]

In [28]:
len(df_fp)

216

In [29]:
df_fp.head() 

,comment_text,Matches_isin_str
2,@USER: Man kann natürlich immerzu dieselbe Sau...,"alte, amigo, esel, hund, hunde, linke, sau"
6,"@MEDIUM ...stimmt so nicht ganz: bespiel ""klei...","alte, alter"
8,Warum man diesen Typen einlädt wird sich mir n...,"affe, affen, assi, dümmste, möchtegern, pack, ..."
17,@USER Pest und Cholera.,pest
25,Abgesehen davon werden alte Autoreifen als Bre...,"alte, schmutz"


In [ ]:
df_fp.to_csv("DIKIsmall_FalsePositives.csv", index=None, encoding="utf-8")

In [30]:
#False Negatives
df_fn = df_test[["comment_text"]][df_test["Sub1_Toxic"] > df_test["label_isin_dich"]] 

In [31]:
len(df_fn)

139

In [32]:
df_fn.head()

,comment_text
0,@USER Sie würden wahrscheinlich auch einen Kri...
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...
13,Großen Respekt wie Herr Hallervorden mit der S...
24,Sry aber Preetz hat nicht viel beizutragen. Er...
28,@USER 👍🏻dem kann ich nur zustimmen. Was nützt ...


In [ ]:
df_fn.to_csv("/content/gdrive/My Drive/Uncivil Annotations/Incivility Dictionary Full Paper/Analyse/DIKIsmall_FalseNegatives.csv", index=None, encoding="utf-8")